#Topic modeling with top2vec

## Import libraries

In [3]:
!pip install top2vec
!pip install top2vec[sentence_encoders]
#!pip install emot
!pip install demoji
!pip install emoji
!conda install tbb
!pip install --upgrade tbb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.2 MB 7.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 7.4 MB/s 
     |████████████████████████████████| 24.1 MB 102.2 MB/s 
     |████████████████████████████████| 1.1 MB 54.3 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2340268 sha256=ef99fb0fa78d752ed8272ac4530d7d51072581a751e882953f547da1a683b0da
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef4a71e896b62f5161f2bc8a14c97a
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=a13c7ebf4192004ba74151be6d4653fdc44bb9257eb7a2cecae1cc1ca1b0d072
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Crea

In [4]:
from top2vec import Top2Vec

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import emoji
import demoji
demoji.download_codes()

import json
import pandas as pd
import numpy as np
import regex as re
import pickle

import sys
import os

import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  # Remove the CWD from sys.path while we load stuff.


In [5]:
#Connnect to google drive 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
## Import librabries

py_file_location = "/content/gdrive/MyDrive/NLP-exam"
sys.path.append(os.path.abspath(py_file_location))

import tweet_preprocessing as tp

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/content/gdrive/MyDrive/NLP-exam/tweet_preprocessing.py:20: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
(105482, 6)
(107587, 6)
Original shape:(213069, 6)


/content/gdrive/MyDrive/NLP-exam/tweet_preprocessing.py:145: FutureWarning: The default value of regex will change from True to False in a future version.
  df.Tweet = df.Tweet.str.replace(regex_list[i], replace_list[i])


After cleaning tweets:(213069, 10)
After handling dulicates and nans:(78542, 10)


,Tweet,Date,User,Type,PostID,InclKey,Hashtags,User_tags,Linked_to,RT,Tokenized_tweets
5500,fact checking always biased misleading created...,2021-04-23,KidKool4U,status,1386384766665216001,plandemic,NaN,[@FelixtheAXE],NaN,True,"[fact, checking, always, biased, misleading, c..."
5501,nothing amorphous thing orchestrated happen re...,2021-04-23,georgemas0nn,status,1386383648254054400,plandemic,"[#amorphous, #orchestrated, #agenda, #organic]",[@WenMoonTho],NaN,True,"[nothing, amorphous, thing, orchestrated, happ..."
5502,gotta research vaccine mask entire,2021-04-23,PowerSoula,status,1386382883254853636,plandemic,[#Plandemic],"[@Kareim30184380, @CryptoMessiah]",NaN,False,"[gotta, research, vaccine, mask, entire]"
5503,bill gate practice genocide africa absorb vita...,2021-04-23,dsyunior,status,1386382415380180992,plandemic,"[#Africa, #Plandemic, #EndGame]",NaN,[https://t.co/oV2To6KrEn],False,"[bill, gate, practice, genocide, africa, absor..."
5504,garbage click bait piece torched else tired fa...,2021-04-23,NVPatriotGirl,status,1386381145135144962,plandemic,"[#ufc, #UFC261, #fakenewsmedia, #propaganda, #...","[@reviewjournal, @danawhite]",[https://t.co/ILLtYkbBXC],False,"[garbage, click, bait, piece, torched, else, t..."
5505,wonder many oscar take home tonight,2021-04-23,ethan_douglas,status,1386378505236135937,plandemic,NaN,NaN,[https://t.co/SEopHdINWX],False,"[wonder, many, oscar, take, home, tonight]"
5506,covid used take away people right freedom powe...,2021-04-23,NoConclusion79,status,1386378282707390464,plandemic,NaN,NaN,[https://t.co/2ksHRgprP9],False,"[covid, used, take, away, people, right, freed..."
5507,look proof worldwide scam year test sold world,2021-04-23,JAZZYW257,status,1386378169549209601,plandemic,NaN,NaN,[https://t.co/xmS000Mwjj],False,"[look, proof, worldwide, scam, year, test, sol..."
5508,family member daycare working month child snif...,2021-04-23,ganais_anais,status,1386377427019051014,plandemic,"[#SlipperySlope, #COVID19, #Plandemic]","[@markjam88463456, @existentialrest, @shocking...",NaN,False,"[family, member, daycare, working, month, chil..."
5509,hoax boris struggle sell longer,2021-04-23,Fifi_Borgia,status,1386377425358147584,plandemic,NaN,[@darylpeagram],NaN,True,"[hoax, boris, struggle, sell, longer]"


## Load and preprocess tweets

In [10]:
#create plandemic and scamdemic list of subdirecvtories
b = "\'1WwYhT1PPii8x-O4GLnMm54E9DnMrcTED\'"
plandemic_subfolders = tp.matching_items_in_directory(b, 'plandemic')
scamdemic_subfolders = tp.matching_items_in_directory(b, 'scamdemic')

#clean and create dataset for each context
df_plandemic = tp.clean_combine_subfolder_tweets(plandemic_subfolders)
df_scamdemic = tp.clean_combine_subfolder_tweets(scamdemic_subfolders)

#separate dataframes
print(df_plandemic.shape)
print(df_scamdemic.shape)

#combined dataframes
df = pd.concat([df_plandemic, df_scamdemic], ignore_index=True)
display(df)

(105482, 6)
(107587, 6)


,Tweet,Date,User,Type,PostID,InclKey
0,RT @libertytarian: 1st day of pastor's trial f...,2021-04-30,SyrianArchangle,status,1390250830956318721,plandemic
1,RT @sjaanie123: Kijk uit voor de gevaccineerde...,2021-04-30,SaskiaDev,status,1390250693534097408,plandemic
2,@Bameron2 You’re boyfriend must be proud of yo...,2021-04-30,Phoenix1783,status,1390250538026086400,plandemic
3,Kijk uit voor de gevaccineerden mensen! #COVID...,2021-04-30,sjaanie123,status,1390249938118905856,plandemic
4,"RT @BartosVl: Covid inženýři z celého světa, j...",2021-04-30,Lubomr12663547,status,1390249162176974848,plandemic
...,...,...,...,...,...,...
213064,"RT @SJJB55: Sept 2020, sat in our village (pop...",2021-04-21,mherbert617,status,1384048634195431432,scamdemic
213065,@BhupinderJaryal This is ..scamdemic,2021-04-21,SharmaSandeep41,status,1384047994165628932,scamdemic
213066,"RT @SJJB55: Sept 2020, sat in our village (pop...",2021-04-21,louisrossilevy,status,1384047719354753025,scamdemic
213067,"Guess what , nobody's gonna get the money 🤣😂\n...",2021-04-21,DanielScoland,status,1384047365724663810,scamdemic


In [11]:
def preprocess_tweets(df):
  ''' Calling preprocessing steps to handle tweets before it is fed into model. '''
  df = tp.emojis_to_text(df)
  df = tp.extract_tweet_features(df)
  df = tp.clean_tweets(df)
  df = tp.handle_duplicates_nans(df)
  df.Tweet = df.Tweet.str.replace(r'\b(\w{1,2})\b', '')

  return df

df = preprocess_tweets(df)
display(df)

/content/gdrive/MyDrive/NLP-exam/tweet_preprocessing.py:145: FutureWarning: The default value of regex will change from True to False in a future version.
  df.Tweet = df.Tweet.str.replace(regex_list[i], replace_list[i])
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


,Tweet,Date,User,Type,PostID,InclKey,Hashtags,User_tags,Linked_to,RT
40,how about shop classes son high school shop...,2021-04-30,GingrichVera,status,1390227188851556353,plandemic,NaN,[@tr6_guy],NaN,False
138,smell meltdown biblical proportions the w...,2021-04-30,Emm_Down_Under,status,1390186145410031616,plandemic,NaN,[@CailinGaeilge8],NaN,True
141,guess what the bandwagon tweeps are pushing b...,2021-04-30,NICHONASRl,status,1390185828236861440,plandemic,[#LifeBeforeCovid],[@njenga_richard],NaN,True
162,herdachten mei huidige regeringsleiders gaan...,2021-04-30,mirakelwolf,status,1390171838651715595,plandemic,NaN,[@kasleuk],NaN,True
173,same situation here india not single calle...,2021-04-30,koshe_suhas,status,1390161332872257539,plandemic,"[#plandemic, #Genocide]",[@jeffsekerak],NaN,False
...,...,...,...,...,...,...,...,...,...,...
213038,asymptomatic spread not thing based lies ...,2021-04-21,Eamonn_75,status,1384061311034613765,scamdemic,"[#Scamdemic, #JUSTSAYNO]",NaN,[https://t.co/nUgawjtqTT],False
213039,make our own decisions about what safe no...,2021-04-21,DIYTips4,status,1384061176888193029,scamdemic,NaN,[@MattHancock],NaN,False
213053,she disgrace and another one that relys th...,2021-04-21,LondonL04023649,status,1384053243387338752,scamdemic,[#scamdemic],[@Valkyrie20201],NaN,False
213059,covid19 artificial coronavirus made france ...,2021-04-21,dav_s_,status,1384051544161611780,scamdemic,[#Scamdemic],NaN,[https://t.co/RJdtezCgOJ],False


In [12]:
df.shape

(78542, 10)

## Top2Vec Model

Credit and documentation:
* https://top2vec.readthedocs.io/en/latest/Top2Vec.html#example 
* https://top2vec.readthedocs.io/en/latest/api.html 
* https://towardsdatascience.com/how-to-perform-topic-modeling-with-top2vec-1ae9bb4e89dc

In [14]:
tweet_list = df['Tweet'].tolist()
model = Top2Vec(tweet_list, embedding_model='universal-sentence-encoder-multilingual', min_count = 5)

#saving the model
model.save("T2V_twitter")
pickle.dump(model, open("/content/gdrive/MyDrive/NLP-exam//t2v.pkl", "wb"))

In [16]:
import pickle

#loading model
#model = Top2Vec.load("T2V_twitter")
model = pickle.load(open("/content/gdrive/MyDrive/NLP-exam//t2v.pkl", "rb"))

In [17]:
n_topics = model.get_num_topics()
topic_sizes, topic_nums = model.get_topic_sizes()
topic_words, word_scores, topic_nums = model.get_topics()

In [23]:
print('Number of topics:', n_topics)
print('Number of words per topic:', topic_sizes)

Number of topics: 315
Number of words per topic: [3816 2547 1955 1505 1377 1330 1207 1140 1133 1070  938  895  879  836
  806  736  730  719  702  681  672  671  653  634  615  611  610  602
  601  571  569  558  530  520  519  510  482  480  467  445  436  435
  435  434  433  428  418  415  405  398  395  391  390  389  385  383
  380  379  377  372  363  361  359  356  352  348  346  321  319  317
  315  308  305  299  297  290  288  283  279  277  276  265  258  257
  253  249  246  246  242  233  229  229  227  225  224  223  221  220
  216  214  213  212  211  207  205  203  202  201  199  198  194  193
  191  182  181  181  180  180  179  178  177  177  176  175  174  173
  173  173  173  172  171  170  169  166  166  164  163  163  163  163
  161  160  160  159  158  158  157  157  154  153  152  151  150  150
  149  146  145  144  144  143  143  142  142  139  139  138  136  135
  135  134  134  133  132  131  129  127  126  126  125  121  119  119
  118  117  116  116  115  1

## Reduced topics

In [25]:
model.hierarchical_topic_reduction(num_topics=12)

[[25,
  116,
  101,
  113,
  132,
  22,
  15,
  73,
  26,
  20,
  62,
  195,
  61,
  79,
  60,
  118,
  122,
  221,
  252,
  256,
  0],
 [203,
  18,
  57,
  84,
  211,
  281,
  243,
  278,
  87,
  112,
  40,
  68,
  224,
  124,
  143,
  33,
  240,
  4,
  81,
  74,
  17,
  192,
  69,
  276,
  123,
  34,
  244,
  13,
  249,
  153,
  222,
  279,
  30,
  95,
  288,
  139,
  41,
  23,
  94,
  265,
  64,
  43,
  248,
  39,
  151,
  51,
  125,
  152,
  180,
  241,
  307,
  108,
  190,
  66,
  312,
  48,
  169,
  214,
  158,
  305,
  189,
  107,
  114,
  147,
  197,
  77],
 [93,
  253,
  14,
  232,
  89,
  235,
  272,
  117,
  273,
  174,
  96,
  110,
  188,
  194,
  217,
  213,
  233,
  257,
  287,
  1],
 [133,
  271,
  5,
  260,
  126,
  49,
  263,
  228,
  65,
  245,
  38,
  207,
  269,
  154,
  295,
  140,
  209,
  187,
  206,
  145,
  56,
  121,
  10],
 [102,
  184,
  205,
  47,
  31,
  9,
  210,
  298,
  109,
  179,
  6,
  160,
  171,
  264,
  11,
  80,
  146,
  100,
  105,
  70,
  88,
 

In [ ]:
#Number of topics that the reduced topics are composed of
hierarchy = model.get_topic_hierarchy()

for i in range(len(hierarchy)):
  print("Reduced topic {0} composed of {1} initial topics".format(i+1, len(hierarchy[i])))

Reduced topic 1 composed of 21 initial topics
Reduced topic 2 composed of 66 initial topics
Reduced topic 3 composed of 20 initial topics
Reduced topic 4 composed of 23 initial topics
Reduced topic 5 composed of 24 initial topics
Reduced topic 6 composed of 37 initial topics
Reduced topic 7 composed of 25 initial topics
Reduced topic 8 composed of 28 initial topics
Reduced topic 9 composed of 9 initial topics
Reduced topic 10 composed of 15 initial topics
Reduced topic 11 composed of 23 initial topics
Reduced topic 12 composed of 24 initial topics


In [ ]:
#Number of words for each reduced topic
n_topics_reduced = model.get_num_topics(reduced=True)
topic_sizes_reduced, topic_nums_reduced = model.get_topic_sizes(reduced=True)

print('Number of topics after hierarchical reduction: {0} \n'.format(n_topics_reduced))
for size, num in zip(topic_sizes_reduced, topic_nums_reduced): 
  print(f"Topic {num+1} with {size} number of words")

Number of topics after hierarchical reduction: 12 

Topic 1 with 9028 number of words
Topic 2 with 8411 number of words
Topic 3 with 7595 number of words
Topic 4 with 7474 number of words
Topic 5 with 6977 number of words
Topic 6 with 6462 number of words
Topic 7 with 6444 number of words
Topic 8 with 6087 number of words
Topic 9 with 5806 number of words
Topic 10 with 4993 number of words
Topic 11 with 4748 number of words
Topic 12 with 4517 number of words


In [ ]:
# Words associated with each reduced topic
topic_words_reduced, word_scores_reduced, topic_nums_reduced = model.get_topics(reduced=True)

for word, score, num in zip(topic_words_reduced, word_scores, topic_nums_reduced): 
  print(f"Topic: {num+1} \n {word} \n {score}\n")

Topic: 1 
 ['vaccinedeaths' 'vaccinekills' 'vaccines' 'vaccineswork' 'vaccinedeath'
 'vaccini' 'vaccins' 'vaccineskill' 'vaccineforall' 'vaccine' 'vaccin'
 'vaccino' 'vaccinatie' 'vaccinations' 'vaccinating' 'vaccinate'
 'vaccineinjury' 'vaccination' 'vaccineagenda' 'vaccinated' 'vacunas'
 'vaccinedamage' 'vaccinesafety' 'vacunaci' 'unvaccinated' 'vacuna'
 'vaccinerollout' 'getvaccinated' 'vaksin' 'vaccinepassport' 'vaxxers'
 'antivaccine' 'vaxxer' 'vaksinasi' 'pfizervaccine' 'vaxxed' 'vaxxing'
 'unvaxxed' 'vaxxes' 'vaxxines' 'szczepionki' 'covidvaccines' 'vaxxine'
 'antivaxxers' 'immunologist' 'vaxzealots' 'covidvaccine' 'antivaxxer'
 'itsnotavaccine' 'vaxx'] 
 [0.6774769  0.6744602  0.67332333 0.672708   0.671281   0.6700614
 0.6686387  0.66820216 0.6647178  0.6647171  0.66274536 0.66146463
 0.65954816 0.6545212  0.6532632  0.65056974 0.6427827  0.6390305
 0.6208806  0.6163132  0.61064845 0.6094098  0.6064526  0.59249055
 0.5918697  0.5891125  0.56465274 0.5575556  0.5547306  0.55359

In [ ]:
import math

topic_sizes, topic_nums = model.get_topic_sizes()
print(topic_sizes[(math.floor(len(topic_nums)/2))]) #median topic size
print(topic_sizes)

144
[3816 2547 1955 1505 1377 1330 1207 1140 1133 1070  938  895  879  836
  806  736  730  719  702  681  672  671  653  634  615  611  610  602
  601  571  569  558  530  520  519  510  482  480  467  445  436  435
  435  434  433  428  418  415  405  398  395  391  390  389  385  383
  380  379  377  372  363  361  359  356  352  348  346  321  319  317
  315  308  305  299  297  290  288  283  279  277  276  265  258  257
  253  249  246  246  242  233  229  229  227  225  224  223  221  220
  216  214  213  212  211  207  205  203  202  201  199  198  194  193
  191  182  181  181  180  180  179  178  177  177  176  175  174  173
  173  173  173  172  171  170  169  166  166  164  163  163  163  163
  161  160  160  159  158  158  157  157  154  153  152  151  150  150
  149  146  145  144  144  143  143  142  142  139  139  138  136  135
  135  134  134  133  132  131  129  127  126  126  125  121  119  119
  118  117  116  116  115  114  114  113  112  111  110  109  109  108
  

In [ ]:
print(topic_words_reduced)

[['vaccinedeaths' 'vaccinekills' 'vaccines' 'vaccineswork' 'vaccinedeath'
  'vaccini' 'vaccins' 'vaccineskill' 'vaccineforall' 'vaccine' 'vaccin'
  'vaccino' 'vaccinatie' 'vaccinations' 'vaccinating' 'vaccinate'
  'vaccineinjury' 'vaccination' 'vaccineagenda' 'vaccinated' 'vacunas'
  'vaccinedamage' 'vaccinesafety' 'vacunaci' 'unvaccinated' 'vacuna'
  'vaccinerollout' 'getvaccinated' 'vaksin' 'vaccinepassport' 'vaxxers'
  'antivaccine' 'vaxxer' 'vaksinasi' 'pfizervaccine' 'vaxxed' 'vaxxing'
  'unvaxxed' 'vaxxes' 'vaxxines' 'szczepionki' 'covidvaccines' 'vaxxine'
  'antivaxxers' 'immunologist' 'vaxzealots' 'covidvaccine' 'antivaxxer'
  'itsnotavaccine' 'vaxx']
 ['despicable' 'scumbag' 'fakenews' 'scumbags' 'covidhoax' 'deplorable'
  'nefarious' 'arrestfaucinow' 'fuckcovid' 'arrestfauci' 'bullshit'
  'propagandists' 'disgrace' 'idiocracy' 'cowards' 'cucked' 'foxnews'
  'fuck' 'disgraceful' 'hoaxed' 'scandals' 'perpetrated' 'lmaooo'
  'disgraced' 'dipshit' 'irony' 'davidicke' 'hoax' 'arre

In [ ]:
print(word_scores_reduced)

[[0.59121937 0.5776341  0.5730162  0.571877   0.57088304 0.5707607
  0.56898284 0.5657487  0.5629636  0.5627527  0.5588828  0.5578666
  0.5562085  0.5524488  0.5523565  0.5515797  0.54590917 0.5375673
  0.529333   0.5262833  0.51564085 0.51364547 0.50697684 0.50052935
  0.49748638 0.4974039  0.49230465 0.48849314 0.48176464 0.47062564
  0.47043598 0.4563875  0.4554856  0.44383    0.43191352 0.4296761
  0.42775822 0.42457747 0.4160748  0.41251427 0.41136464 0.4062687
  0.40522406 0.40022993 0.3983243  0.39252335 0.38947916 0.38906473
  0.38796026 0.3858899 ]
 [0.32805505 0.31990272 0.3194555  0.31474826 0.31196153 0.31155205
  0.3110633  0.30444026 0.30327004 0.3002233  0.29690206 0.2965939
  0.29492787 0.2946611  0.2932521  0.29198068 0.29142618 0.29096723
  0.29072732 0.29044014 0.28977174 0.28929123 0.2892908  0.28875023
  0.28862405 0.28807703 0.28657895 0.28656992 0.2832551  0.28311706
  0.2823938  0.28211585 0.28153735 0.28005442 0.27957982 0.27869606
  0.27811038 0.27779114 0.277

In [ ]:
for word, num in zip(topic_words_reduced, topic_nums_reduced): 
  words = word[:20]
  print(f"Topic {num}: {', '.join(words)}")

Topic 0: vaccinedeaths, vaccinekills, vaccines, vaccineswork, vaccinedeath, vaccini, vaccins, vaccineskill, vaccineforall, vaccine, vaccin, vaccino, vaccinatie, vaccinations, vaccinating, vaccinate, vaccineinjury, vaccination, vaccineagenda, vaccinated
Topic 1: despicable, scumbag, fakenews, scumbags, covidhoax, deplorable, nefarious, arrestfaucinow, fuckcovid, arrestfauci, bullshit, propagandists, disgrace, idiocracy, cowards, cucked, foxnews, fuck, disgraceful, hoaxed
Topic 2: democrats, electionfraud, voterfraud, dems, trumpwasright, republicans, liberals, reelected, politicised, cucked, democrat, demoncrats, libtards, fearmongering, electorate, unelected, conservatives, election, dictators, trumps
Topic 3: bullshit, scumbag, yawn, bastards, fuck, hahahahaha, lmao, fuckfauci, scumbags, dipshit, dumbass, hahahahahaha, goddamn, furiously, lmaooo, nefarious, dipshits, idiots, idiocy, wow
Topic 4: virushoax, epidemic, vaccinedeaths, epidemics, vaccinedeath, coronavirushoax, influenza, p

## Wordclouds

In [ ]:
from wordcloud import WordCloud
from google.colab import files

wc = WordCloud(background_color='white', width = 500, height = 250, margin=4, contour_width=3, contour_color='steelblue', min_font_size = 7)

# Words associated with each reduced topic
topic_words_reduced, word_scores_reduced, topic_nums_reduced = model.get_topics(reduced=True)

for num in range(len(topic_nums_reduced)):
  top_dict = dict(zip(topic_words_reduced[num], word_scores_reduced[num]))

  a = wc.fit_words(top_dict)
  plt.figure(figsize=[15,10])
  plt.axis("off")
  plt.title(''.join(['Topic ', str(num)]), fontsize = 40)
  plt.imshow(a)
  plt.show()
  
  filename = ''.join(['wc', str(num), '.png'])
  plt.savefig(filename)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#wordclouds
for topic in topic_nums_reduced: 
  model.generate_topic_wordcloud(topic, reduced=True)

Output hidden; open in https://colab.research.google.com to view.

## Similar keywords

In [ ]:
words, word_scores = model.similar_words(keywords=["vaccine"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

vaccin 0.9924864243189836
vaccinate 0.9870499641107221
vaccini 0.9816555706012706
vaccino 0.981192370357928
vaccinating 0.9757630898549879
vaccination 0.9720793282676994
vaccinatie 0.9689491994050952
vaccinated 0.9611254452679566
vaccineswork 0.9599617184138713
vaccines 0.9577330715057207
vaccins 0.9573394263483102
vaccinations 0.9440655116838395
vacuna 0.9295061062511403
vacunaci 0.9293636099470519
vaccinedamage 0.9270081714975223
vaccineinjury 0.9229744448750938
vacunas 0.9151144857686917
vaccineforall 0.9045098736574794
vaccineskill 0.9034603492160657
vaccinekills 0.900132476035136


In [ ]:
words, word_scores = model.similar_words(keywords=["vaccinekills"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

vaccineskill 0.9799539380132839
vaccineswork 0.9237192149693676
vaccinedeaths 0.9228182576805664
vaccines 0.9177866419786911
vaccins 0.912548422582454
vaccini 0.9101393191287683
vaccinations 0.9080025737575325
vaccine 0.9001324397653248
vaccinedeath 0.8990080037772187
vaccinatie 0.8955516811939072
vaccineinjury 0.8883530382305722
vaccineforall 0.887608150169104
vaccinating 0.8863363725409341
vaccinate 0.8851508493213975
vaccino 0.8843039241760589
vaccin 0.8841437906228296
vaccination 0.8803264433866571
vaccinedamage 0.8704711239592872
vaccineagenda 0.8642454037095373
vacunas 0.8635616075055338


In [ ]:
words, word_scores = model.similar_words(keywords=["vaccineagenda"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

vaccinatie 0.9090242026823154
vaccination 0.9018780017136692
vaccineswork 0.90168871584808
vaccinedamage 0.8975998514271193
vaccino 0.8965782865124721
vaccini 0.8964208704884741
vaccine 0.8960078252040676
vaccinate 0.8916236953999075
vaccinating 0.8880959412385031
vaccineinjury 0.8845338663965903
vaccin 0.8744375719998712
vaccinations 0.8736860368119861
vaccinedeaths 0.873402930675234
vaccinated 0.8723696343040772
vaccinedeath 0.8722526933253553
vaccineforall 0.87070985706344
vaccines 0.8701274350385478
vaccineskill 0.8649433079636548
vaccinekills 0.8642454511969555
vacunaci 0.8609873407653206


In [ ]:
words, word_scores = model.similar_words(keywords=["truthers"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

truther 0.9529460367418787
truths 0.8887342565407297
truthwins 0.8863388021563676
truthmatters 0.854877463176871
truth 0.8400378830901064
truthful 0.8228530419828686
waarheid 0.7533365889668742
truthmusic 0.7082267299510209
verdade 0.6641737990631872
verdad 0.6591919933028138
falsehoods 0.6454541983218216
liars 0.6384977862294198
honest 0.6367479768815678
honesty 0.6299318664597837
factual 0.6267134850779333
liar 0.6260390135097709
fauxi 0.6234534124157936
lie 0.6216184243200014
facts 0.6130401539477057
fact 0.6121988205429318


In [ ]:
words, word_scores = model.similar_words(keywords=["fauci"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

faucis 0.8640378059050946
faucigate 0.8477606429964688
fau 0.8015320043875316
tou 0.762238423700883
maak 0.7474012193065673
fuk 0.7455296586332754
gof 0.7384180159692495
faucilies 0.7368336774788906
ken 0.7340813178140517
fella 0.732346375158136
keen 0.7302341216424837
bla 0.730171690612581
faucilied 0.7264545766109957
enuf 0.7248021266607317
fucker 0.7234199170877469
phe 0.7229121861351915
laat 0.7225695304555191
dear 0.7223270577037159
sniffle 0.7210813947156776
tony 0.7210460230549736


In [ ]:
words, word_scores = model.similar_words(keywords=["mask"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

masks 0.9614005583956569
masked 0.9099247619688073
masking 0.9007316810176095
masksoff 0.8728193611583888
maskoff 0.8675209043897705
maskless 0.8608374480848502
unmask 0.8549450132120289
maskmandates 0.8515292346447045
maskhole 0.8488013385049831
maskface 0.848794183351541
unmasking 0.8393437216907591
masker 0.8319504858709748
maskmandate 0.8260273988827194
masquerade 0.8190368649418862
maskskill 0.8077508616985355
wearamask 0.8043620842122581
masquerading 0.8007437966913704
maskholes 0.7979867680875055
maskprohibited 0.7977223771138018
fuckmasks 0.7920666287822914


In [ ]:
words, word_scores = model.similar_words(keywords=["fuckmasks"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

fuckyourmask 0.9156242670933906
unmasking 0.8685595598432236
nomasks 0.8612368964676487
takeoffthemasks 0.8486911843407056
fcukmasks 0.8413348300029038
wearamask 0.8393266395110911
unmask 0.8335918680263408
unmasked 0.82499738556804
nomoremasks 0.8231611053581966
masks 0.8158882499260327
masksoff 0.8111135803065737
saynotomasks 0.8048259367278074
nomask 0.8010466161022247
maskmandates 0.8010372921118234
mask 0.7920666309240102
facemasks 0.7810746002560508
takeoffyourmask 0.7790139089883966
maskless 0.7739863390172024
maskprohibited 0.7685769470609638
maskskill 0.7675581939940611


In [ ]:
words, word_scores = model.similar_words(keywords=["government"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

governmental 0.975684735456195
regering 0.9575957648027384
governments 0.9575071332222636
pemerintah 0.9447865405881399
gov 0.9287857680622157
overheid 0.9261517210570811
govs 0.9213094545561129
govts 0.9081934315946083
govt 0.8986868823715415
ukgovernment 0.8568177821536369
govern 0.8497869342520992
govdid 0.7902057281434577
governing 0.7865552929352317
governance 0.7709308110197579
authorities 0.7308923629465228
administration 0.7049231120715314
goverment 0.6998674139559435
negara 0.6958537887038434
goverments 0.6733348396014582
governors 0.6732117860845817


In [ ]:
words, word_scores = model.similar_words(keywords=["death"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

muerte 0.9335476258233666
kematian 0.9218878247309408
dying 0.8778761365362675
deaths 0.8696833061774588
dieing 0.8620106335388574
muertes 0.8546981157147417
deathly 0.8506058153113206
mati 0.8238611420627212
mortality 0.8213217247923268
dead 0.8170236322955368
dies 0.811869488165549
died 0.8040761529346259
killed 0.7698808393761192
killing 0.7529897083863745
fatality 0.7498963894162674
deceased 0.7485502078571414
meninggal 0.745645667409828
demise 0.7388498872504696
fatal 0.7360736931414846
kill 0.7331247303963186


In [ ]:
words, word_scores = model.similar_words(keywords=["demic"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

fakedemic 0.8208469972301841
scaredemic 0.8161230933834505
pandemic 0.8144575688004921
sparspandemic 0.7685705332425172
feardemic 0.7652763422364365
hoaxdemic 0.7615509558957342
testdemic 0.7602012622124856
scamedemic 0.758058992073784
panicdemic 0.7503762567196826
scandemic 0.7450247055481656
infodemic 0.7433939063389365
casedemic 0.7422700676374698
covidpandemic 0.7372672440425914
endthepandemic 0.7231378414963316
plannedemic 0.7206097102211213
condemic 0.7112397973980574
planneddemic 0.7024193713486307
fakepandemic 0.7024139741077566
cyberpandemic 0.7012231033354627
shamdemic 0.694942616281947


In [ ]:
words, word_scores = model.similar_words(keywords=["trumpwasright"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

trump 0.6408966905695743
trumps 0.5911143713986158
trumpers 0.5872189016425672
trumpwon 0.582433150834412
presidenttrump 0.5375157368531365
cucked 0.5343733897352653
donaldtrump 0.5273967180190899
deplorable 0.5264559043672289
unironically 0.49693848575404087
canceltrumpism 0.489946992582206
leftist 0.48418065794586285
downright 0.47069106681904405
legitimately 0.469630948852544
fakepresident 0.46935643731884935
justified 0.46895250175021064
dumbass 0.46882414047498816
shuttered 0.4686821864683374
nazi 0.4686627925420128
rightfully 0.4666921192952741
liberals 0.46651239587354376


In [ ]:
words, word_scores = model.similar_words(keywords=["electionfraud"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

voterfraud 0.8846624762880271
election 0.7464164836040985
elections 0.7265332241166528
pcrfraud 0.6994519656018212
fraudci 0.688971167596793
electoral 0.6854766846055002
pandemicfraud 0.6769890658583951
faucithefraud 0.6749232267231715
stolenelection 0.6730467593844165
fraudchi 0.6692066999811532
fraude 0.6667561068818674
fraud 0.6663378540734268
frauds 0.666082972842148
fraudster 0.6660465541829796
electorate 0.6640159124900191
pcrtestfraud 0.661558145094328
fraudsters 0.6609018757714011
faucifraud 0.6585991557752287
fraudemia 0.6543796778589045
fraudulent 0.653149715696451


In [ ]:
words, word_scores = model.similar_words(keywords=["election"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

elections 0.9583177635891285
electoral 0.865836381093213
elected 0.851959103080361
elect 0.8169421229100781
voters 0.7945530160486198
electorate 0.7854797358350711
voter 0.7808111865472176
candidate 0.7596363359419918
electionfraud 0.7464165397435032
reelected 0.7431714008635928
candidates 0.7398526034133989
voting 0.7395989695088254
ballots 0.7389782140354146
ballot 0.7193347015166536
presidency 0.7168043577279326
vote 0.7117581151740209
votes 0.6847505119531044
presidential 0.6784196823147081
politics 0.6666289308564068
stolenelection 0.6638369994518742


In [ ]:
words, word_scores = model.similar_words(keywords=["democrats"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

democrat 0.9345430108614902
republicans 0.8645433021026134
dems 0.8179252401188868
republican 0.812531426087544
democratic 0.7848283038194314
liberals 0.7199894990141137
voters 0.7026344383676209
democracy 0.6955028332918589
trumps 0.6930607631297137
politicos 0.6896862163295887
conservatives 0.6696411980963277
trumpers 0.660076066367228
demoncrats 0.657055513549172
democracies 0.6555836645820642
politicians 0.6552531568464894
liberal 0.648919109051298
libs 0.647993592262067
trump 0.6475905361230847
politico 0.6366566944820026
leftists 0.6329542326100674


In [ ]:
words, word_scores = model.similar_words(keywords=["republicans"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

republican 0.9333362946741368
democrats 0.8645433045937483
democrat 0.7693361116890418
liberals 0.7376075501624111
dems 0.7310513964099182
conservatives 0.7158702364037663
politicos 0.6925235996019377
trumps 0.682167489708383
trumpers 0.6586520043416104
libs 0.6550726138189849
voters 0.6507948664516652
politicians 0.6463320165957733
liberal 0.6433295461622703
partisan 0.6333768463750199
politico 0.6199486152709326
libtards 0.6194153272202179
trump 0.6165198757713439
senators 0.6125750748173011
leftists 0.6038754909803705
presidents 0.6037211751046707


In [ ]:
words, word_scores = model.similar_words(keywords=["biden"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

bidens 0.9662785823316726
bidensamerica 0.8006566164025836
doden 0.6443431606538293
dems 0.5817243592933299
pence 0.5643726772906342
politician 0.5559334597722401
stooge 0.5546045410071686
voters 0.5533604069242753
twit 0.5530846716183351
pres 0.5510831891739271
electorate 0.5504309995054097
debacle 0.5479559292269673
senator 0.5454607852398989
ballots 0.5452392664445531
president 0.5445281083850428
bihar 0.5440227483484197
obummer 0.5430540132303674
crook 0.5426234126043638
whitehouse 0.5425410401214843
denies 0.5418531276355772


In [ ]:
words, word_scores = model.similar_words(keywords=["trump"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

trumps 0.8958037206370704
trumpers 0.8140861172631793
presidenttrump 0.728367941611094
donaldtrump 0.7226011747911998
donald 0.7014618523491328
trumpwon 0.6986424031223213
democrat 0.6651338599870243
president 0.6583839232081516
democrats 0.6475905271476392
obama 0.646981694575391
republican 0.6459962914100512
trumpwasright 0.6408966726715334
presidential 0.6395796979490418
presidency 0.63566547359457
clinton 0.6335149588217455
presidents 0.6294013391764555
clintons 0.6217157542079356
republicans 0.6165198654500618
cucks 0.6152317008726568
dems 0.6150216557078093


In [ ]:
words, word_scores = model.similar_words(keywords=["raid"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

loot 0.6514501089591457
defeat 0.6477887431077378
fight 0.6396620591178742
kick 0.6351786816882924
attacked 0.6343157601616751
attack 0.6333204148663636
attacking 0.62959996670352
wrath 0.6289136758175529
strike 0.6281209088556744
kill 0.6276132651398627
kicking 0.6270246216499189
attacks 0.622357898658401
piling 0.621921153853002
havoc 0.6198225475069293
beaten 0.6192127150322977
dashing 0.6179364380250866
defeating 0.6154048737176416
looted 0.6142909525642388
dragged 0.6138294496986405
kicked 0.6133714606205576


In [ ]:
words, word_scores = model.similar_words(keywords=["billionaires"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

millionaires 0.8676543015141194
billionaire 0.799582845518599
millionaire 0.7752562073170617
billions 0.7525118832737872
billion 0.6897618056139385
trillions 0.6603777151969225
richest 0.6444506309872933
trillion 0.6146649352900592
wealthiest 0.6145632444484447
wealth 0.604595915089325
million 0.6043758025317372
millions 0.6012286840586348
wealthier 0.6004152367200761
wealthy 0.5953040025573518
rich 0.591671984710847
oligarchs 0.5864130901687762
oligarchy 0.555754151153083
reich 0.545101884914905
richer 0.5280547104288832
capitalists 0.5240272296137811


In [ ]:
words, word_scores = model.similar_words(keywords=["news"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

berita 0.8739818529749538
newsflash 0.8346017148425033
ottnews 0.7965228588922076
newsmax 0.775480938499445
uknews 0.6917971751100523
fakenews 0.6880706933888506
informa 0.6845799147789317
worldnews 0.6823621585018341
newspapers 0.6819806679511736
headline 0.6752982240145792
headlines 0.6726055585018152
informed 0.6691948059845682
skynews 0.6638446022105174
breakingnews 0.6627153319538779
announces 0.6582311396905699
inform 0.6571306650894033
announcement 0.6550733906489867
informing 0.6520170332983574
reporting 0.6466322039190293
journalism 0.6453019819256047


In [ ]:
words, word_scores = model.similar_words(keywords=["foxnews"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

skynews 0.760058668629799
fakenews 0.7370930775599611
ottnews 0.7184497321489238
fox 0.7088895017454214
fakenewsmedia 0.7056593498753945
breakingnews 0.70012619509783
bbcnews 0.6938085810402675
uknews 0.6836521853597699
worldnews 0.6809781858846775
newsmax 0.6541045764835808
news 0.6427371793551688
berita 0.6411802879112998
globalnews 0.6345504693519293
newsflash 0.6179466651209418
msnbc 0.6124192596603947
hoaxvid 0.5839655079139733
covidhoax 0.5781115168803246
oxpile 0.5718349434993251
fauxvid 0.5657608078919152
cnn 0.5652548200669674


In [ ]:
words, word_scores = model.similar_words(keywords=["cnn"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

glenn 0.722897367793108
dnr 0.6620926272380347
nsw 0.6617962163113265
snd 0.6541740887035632
isn 0.6540008954198021
ngl 0.6539257327937718
dnc 0.6483430266655228
din 0.6449529002983465
sdg 0.6446825581850668
nhs 0.6404295812575318
flynn 0.6390754347298306
lan 0.6369840847030499
attn 0.6349440043359029
dec 0.6318499586227911
shunned 0.6311401598039166
wn 0.6279238328238352
hcq 0.6269051797286151
bil 0.6255655087604142
ctv 0.6253229696881595
nft 0.623294486943132


In [ ]:
words, word_scores = model.similar_words(keywords=["fakenews"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

fakenewsmedia 0.8636025658517387
ottnews 0.7632291606876669
fake 0.7545442944951031
breakingnews 0.7508740592003967
foxnews 0.7370930404042535
faked 0.7248602674631356
berita 0.7002837193827482
uknews 0.6953867345852129
faking 0.6905759956412925
news 0.6880706282819282
fakepresident 0.6741289538259636
newsflash 0.6643355086207636
newsmax 0.6592100974412801
fakery 0.658143058306587
skynews 0.6523675184123917
fakescience 0.6427725416912384
hoaxed 0.6366722884910364
worldnews 0.6351097731318472
fauxi 0.6294002372213467
hoax 0.627990763576534


In [ ]:
words, word_scores = model.similar_words(keywords=["fakescience"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

pseudoscience 0.7577662664927989
science 0.7331224652164352
sciencefraud 0.713625322969943
scientifically 0.7096541898585996
scientific 0.7073122118013522
faking 0.6840295979056787
fake 0.6709790121179272
faked 0.6522789470628542
unscientific 0.6505990659086403
fakenews 0.6427725364492592
scientist 0.617300877401791
scientists 0.613793553606478
fakery 0.5979031326559685
fakedemic 0.5977263169174608
scientism 0.5957189233083754
sceptics 0.5950518064060912
sceptic 0.5902648992627958
hoax 0.5859700874493066
sceptical 0.5846346650084806
faux 0.5813228609121475


In [ ]:
words, word_scores = model.similar_words(keywords=["virushoax"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

coronavirushoax 0.7989732450153704
fakevirus 0.7936778930362385
virussen 0.7916785770786892
virus 0.7646967680598971
viruses 0.7429517012352669
scamronavirus 0.7421053723797951
hoaxdemic 0.7316916204420638
fearvirus 0.7281490979014735
faucivirus 0.724020066956414
chinavirus 0.7236460828902241
covonavirus 0.7168108337781736
hoax 0.7083373536585338
covidhoax 0.6921528729201137
viral 0.6724736720480885
hoaxed 0.6724246146007318
eatthevirus 0.6689790056185967
covidisahoax 0.6650767323095534
wuhanvirus 0.659973375317116
chinesevirus 0.6433186176574239
climatehoax 0.6245201018010103


In [ ]:
words, word_scores = model.similar_words(keywords=["trumpwasright"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

trump 0.6408966905695743
trumps 0.5911143713986158
trumpers 0.5872189016425672
trumpwon 0.582433150834412
presidenttrump 0.5375157368531365
cucked 0.5343733897352653
donaldtrump 0.5273967180190899
deplorable 0.5264559043672289
unironically 0.49693848575404087
canceltrumpism 0.489946992582206
leftist 0.48418065794586285
downright 0.47069106681904405
legitimately 0.469630948852544
fakepresident 0.46935643731884935
justified 0.46895250175021064
dumbass 0.46882414047498816
shuttered 0.4686821864683374
nazi 0.4686627925420128
rightfully 0.4666921192952741
liberals 0.46651239587354376


## Search topics most similar to a given keyword

In [ ]:
topic_words, word_scores, topic_scores_a, topic_nums_a = model.search_topics(keywords=["vaccine"], num_topics=5, reduced=True)
for num, score in zip(topic_nums_a, topic_scores_a):
    print(f"{num+1} {score}")

1 0.5627528652923395
6 0.4625361280651168
5 0.3504511258862383
12 0.20642673471939244
11 0.20389103168475375


In [ ]:
topic_words, word_scores, topic_scores_a, topic_nums_a = model.search_topics(keywords=["mask"], num_topics=5, reduced=True)
for num, score in zip(topic_nums_a, topic_scores_a):
    print(f"{num+1} {score}")

7 0.5177505223221671
6 0.42968880617087823
5 0.19563202731576676
1 0.1925436353129914
11 0.18515895944043304


In [ ]:
topic_words, word_scores, topic_scores_a, topic_nums_a = model.search_topics(keywords=["government"], num_topics=5, reduced=True)
for num, score in zip(topic_nums_a, topic_scores_a):
    print(f"{num+1} {score}")

6 0.4229354679937854
3 0.30507433308011717
12 0.20570864563397764
9 0.2020412854982475
10 0.19852173299437392


In [ ]:
topic_words, word_scores, topic_scores_a, topic_nums_a = model.search_topics(keywords=["demic"], num_topics=5, reduced=True)
for num, score in zip(topic_nums_a, topic_scores_a):
    print(f"{num+1} {score}")

6 0.5686946630926919
5 0.3044471238866494
12 0.2597157838130061
4 0.183061389579262
8 0.14568234636674526


In [ ]:
topic_words, word_scores, topic_scores_a, topic_nums_a = model.search_topics(keywords=["fakedemic"], num_topics=5, reduced=True)
for num, score in zip(topic_nums_a, topic_scores_a):
    print(f"{num+1} {score}")

6 0.5622084537974946
5 0.3486551914523497
8 0.2752559103779376
12 0.2690528490662188
7 0.2312180009760712


In [ ]:
topic_words, word_scores, topic_scores_a, topic_nums_a = model.search_topics(keywords=["scaredemic"], num_topics=5, reduced=True)
for num, score in zip(topic_nums_a, topic_scores_a):
    print(f"{num+1} {score}")

6 0.5711076497693581
5 0.39907123700662095
12 0.29455476899330646
4 0.27152744101611503
7 0.23761537848563863


In [ ]:
topic_words, word_scores, topic_scores_a, topic_nums_a = model.search_topics(keywords=["shamdemic"], num_topics=5, reduced=True)
for num, score in zip(topic_nums_a, topic_scores_a):
    print(f"{num+1} {score}")

6 0.5092285197681119
5 0.310933897678369
4 0.24913768466002303
7 0.24116947887132906
8 0.20384501021439938


In [ ]:
topic_words, word_scores, topic_scores_a, topic_nums_a = model.search_topics(keywords=["epidemic"], num_topics=5, reduced=True)
for num, score in zip(topic_nums_a, topic_scores_a):
    print(f"{num+1} {score}")

6 0.49837028780505616
5 0.4735981372038911
1 0.3812979014009692
11 0.24644245916040497
12 0.23269589343819308


In [ ]:
topic_words, word_scores, topic_scores_a, topic_nums_a = model.search_topics(keywords=["raid"], num_topics=5, reduced=True)
for num, score in zip(topic_nums_a, topic_scores_a):
    print(f"{num+1} {score}")

6 0.5312180593906086
12 0.2858206849356524
4 0.2147138625415219
7 0.18516860053577364
2 0.18049569803728516


## Search document by topic

In [ ]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=4, num_docs=15)
for doc, score, doc_id in zip(documents, document_scores, document_ids): 
  print(f"Document: {doc_id}, Score: {score}")
  print("-----------")
  print(doc)
  print("-----------")
  print()

Document: 34539, Score: 0.8168532848358154
-----------
 millions haven  suffered covid this virus amp its  are over blown let   amp   disappear like all those that before  this     polarize control manipulate amp kill  among other things that should  obvious  now stay aware  conscious amp  kind 
-----------

Document: 37406, Score: 0.804431140422821
-----------
govt turned  pandemic never denied its existence into    amp now the control amp lies amp deceit amp subterfuge are unprecedented amp off the scale 
-----------

Document: 36492, Score: 0.7862367033958435
-----------
this whole   has done nothing but cause  big mess confuse people amp cause people  get sick amp die from their mandates pouting face but that  what the devil does comes  steal kill amp destroy wake  tyranny  the enemy  peace amp safety follow the lord jesus christ
-----------

Document: 66327, Score: 0.7840561270713806
-----------
 was just going  ask why himself amp everyone else  the new left don  care about this 

In [ ]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=3, num_docs=15)
for doc, score, doc_id in zip(documents, document_scores, document_ids): 
  print(f"Document: {doc_id}, Score: {score}")
  print("-----------")
  print(doc)
  print("-----------")
  print()

Document: 62812, Score: 0.7992571592330933
-----------
definitely about  virus look  them best the virus out  the air making sure this guy doesn  get attacked   the world had gone mad   covid19 
-----------

Document: 5771, Score: 0.7874994277954102
-----------
anyone calling the virus the     fucking idiot 
-----------

Document: 20168, Score: 0.7819287180900574
-----------
  what   you  not understanding covid19   real virus  has killed over 580k americans and the cdc isn  making this  there isn      can  wrap  mind around how  many people think this virus  all part  some kind  grand conspiracy grinning squinting face
-----------

Document: 13842, Score: 0.7818622589111328
-----------
 its truly all about the   not the virus
-----------

Document: 1132, Score: 0.7740454077720642
-----------
the virus  the msm fauci cdc who and bill gates once they are eradicated the virus ends   covid  just the flu the media   the super spreader  fear 
-----------

Document: 45807, Score: 0.773539662

In [ ]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=8, num_docs=15)
for doc, score, doc_id in zip(documents, document_scores, document_ids): 
  print(f"Document: {doc_id}, Score: {score}")
  print("-----------")
  print(doc)
  print("-----------")
  print()

Document: 52852, Score: 0.8020060062408447
-----------
fake bullshit   driven  false irrelevant pcr tests stop getting tested end this madness now 
-----------

Document: 31589, Score: 0.772680401802063
-----------
 cases because healthy asymptomatic morons are getting themselves tested with fraudulent pcr tests how  stop     its tracks stop getting tested 
-----------

Document: 10994, Score: 0.7698745727539062
-----------
  have unit stench about   pcr test denounced  tot inaccurate not  used  diagnostic test  man who invented  pcr false positives  how they keep the bogus   going with 1000s  daily false positive tests that are dclared infections2 when  one ill 
-----------

Document: 66439, Score: 0.7676889896392822
-----------
 you  realise those pcr tests are next  useless and mostly give false positives  
-----------

Document: 73090, Score: 0.7529999613761902
-----------
 this just tells  that people are getting unnecessary tested   false pcr test cases mean nothing other than dr

## Semantic Search Documents by Keywords

In [ ]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["vaccineforall"], num_docs=5)

for doc, score, doc_id in zip(documents, document_scores, document_ids): 
  print(f"Document: {doc_id}, Score: {score}")
  print("-----------")
  print(doc)
  print("-----------")
  print()

Document: 41576, Score: 0.9045098325629461
-----------
 vaccine vaccinesideeffects     
-----------

Document: 41777, Score: 0.895319534468062
-----------
  vaccine for    
-----------

Document: 65747, Score: 0.8940633302938117
-----------
     vaccineinjury 
-----------

Document: 65403, Score: 0.8940633302938117
-----------
     vaccineinjury
-----------

Document: 74703, Score: 0.8940633302938117
-----------
 vaccineinjury     
-----------



In [ ]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["maskoff"], num_docs=5)

for doc, score, doc_id in zip(documents, document_scores, document_ids): 
  print(f"Document: {doc_id}, Score: {score}")
  print("-----------")
  print(doc)
  print("-----------")
  print()

Document: 41056, Score: 0.9483907647103765
-----------
 masksoff  
-----------

Document: 18255, Score: 0.819236260536216
-----------
 mask for the   
-----------

Document: 21178, Score: 0.8021577216471197
-----------
 masksoff covid19   
-----------

Document: 27988, Score: 0.7963032983048077
-----------
unmasking 
-----------

Document: 73425, Score: 0.7931766608988955
-----------
     covid19 maskup maskoff 
-----------



In [ ]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["faceclown"], num_docs=5)

for doc, score, doc_id in zip(documents, document_scores, document_ids): 
  print(f"Document: {doc_id}, Score: {score}")
  print("-----------")
  print(doc)
  print("-----------")
  print()

Document: 62273, Score: 0.7745249904707829
-----------
  pouting face 
-----------

Document: 44775, Score: 0.7720707535559098
-----------
  shushing face
-----------

Document: 60054, Score: 0.7703973436401628
-----------
   pleading face 
-----------

Document: 59801, Score: 0.7398316612817291
-----------
   confounded face
-----------

Document: 30462, Score: 0.7356951336565747
-----------
  flushed face
-----------



In [ ]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["facepalmingface"], num_docs=5)

for doc, score, doc_id in zip(documents, document_scores, document_ids): 
  print(f"Document: {doc_id}, Score: {score}")
  print("-----------")
  print(doc)
  print("-----------")
  print()

Document: 41771, Score: 0.8438825749826292
-----------
  man facepalming 
-----------

Document: 18480, Score: 0.8425363596159768
-----------
   person facepalming
-----------

Document: 18481, Score: 0.8425363596159768
-----------
   person facepalming 
-----------

Document: 60054, Score: 0.7938310869931346
-----------
   pleading face 
-----------

Document: 62273, Score: 0.7753252590077011
-----------
  pouting face 
-----------



In [ ]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["fauci"], num_docs=5)

for doc, score, doc_id in zip(documents, document_scores, document_ids): 
  print(f"Document: {doc_id}, Score: {score}")
  print("-----------")
  print(doc)
  print("-----------")
  print()

Document: 30063, Score: 1.0000001043970865
-----------
 fauci coronavirususpatent    2021 
-----------

Document: 26353, Score: 0.8024755343300991
-----------
fauci    done 
-----------

Document: 2731, Score: 0.7290658431441295
-----------
    jago 
-----------

Document: 68432, Score: 0.7264545766109961
-----------
   faucilied 
-----------

Document: 8390, Score: 0.7264545766109961
-----------
   faucilied
-----------



In [ ]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["trumpwasright"], num_docs=5)

for doc, score, doc_id in zip(documents, document_scores, document_ids): 
  print(f"Document: {doc_id}, Score: {score}")
  print("-----------")
  print(doc)
  print("-----------")
  print()

Document: 41062, Score: 0.7104586975266539
-----------
fauci  busted trump was right     
-----------

Document: 13023, Score: 0.6782989836469767
-----------
 trumpgotscrewed   
-----------

Document: 11092, Score: 0.6590000965580298
-----------
    oust trump 
-----------

Document: 71265, Score: 0.6586945341244921
-----------
   trumpgotfleeced 
-----------

Document: 45801, Score: 0.6495151816724793
-----------
 was     hurt the trump administration 
-----------



In [ ]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["electionfraud"], num_docs=5)

for doc, score, doc_id in zip(documents, document_scores, document_ids): 
  print(f"Document: {doc_id}, Score: {score}")
  print("-----------")
  print(doc)
  print("-----------")
  print()

Document: 73364, Score: 1.0000001264025895
-----------
 electionfraud   
-----------

Document: 75357, Score: 1.0000001264025895
-----------
     electionfraud 
-----------

Document: 32121, Score: 0.8016727134247177
-----------
 bifurcatecontrol   electionfraud illegalballotcounting createdby 
-----------

Document: 62958, Score: 0.7799166675378777
-----------
 dementiajoe   electionfraud
-----------

Document: 8144, Score: 0.7700837845486678
-----------
 deep state   election fraud
-----------



In [ ]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["raid"], num_docs=5)

for doc, score, doc_id in zip(documents, document_scores, document_ids): 
  print(f"Document: {doc_id}, Score: {score}")
  print("-----------")
  print(doc)
  print("-----------")
  print()

Document: 60677, Score: 0.6478627632844131
-----------
fight the     worldwidefreedomrally 
-----------

Document: 47866, Score: 0.6396620591178743
-----------
 fight   
-----------

Document: 72553, Score: 0.5773265687443307
-----------
 gop 14daystoflattenthecurbe   
-----------

Document: 73967, Score: 0.5771543987478955
-----------
  spars2025 
-----------

Document: 27650, Score: 0.5720539556595328
-----------
   shout  
-----------



In [ ]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["fakescience"], num_docs=5)

for doc, score, doc_id in zip(documents, document_scores, document_ids): 
  print(f"Document: {doc_id}, Score: {score}")
  print("-----------")
  print(doc)
  print("-----------")
  print()

Document: 16701, Score: 0.7844926917163254
-----------
the real science   
-----------

Document: 49571, Score: 0.7809442197178533
-----------
 propaganda fakescience     
-----------

Document: 23214, Score: 0.7331224652164354
-----------
 science  
-----------

Document: 45755, Score: 0.6936413775634365
-----------
   junk science
-----------

Document: 18939, Score: 0.6709790121179274
-----------
    fake
-----------

